In [ ]:
# Uncomment this to install dependencies for this notebook
# !pip install gym
# !pip install tqdm
# !pip install tensorboard
# !pip install ipympl

In [1]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
import tqdm
import torch.optim as optim


In [2]:
from hypernn.torch.hypernet import TorchHyperNetwork
from hypernn.torch.weight_generator import TorchWeightGenerator
from hypernn.torch.embedding_module import TorchEmbeddingModule

## Basic Hypernetwork

### 3 Components:
- EmbeddingModule: Layer / general parameter embeddings to be passed into the weight generator
- WeightGenerator: Shared network used to generate parameters from each embedding in the EmbeddingModule
- Hypernetwork: Combines EmbeddingModule & WeightGenerator for end to end parameter generator

### EmbeddingModule

In [3]:
from typing import Optional, Any, Dict

class DefaultTorchEmbeddingModule(TorchEmbeddingModule):
    def __init__(self, embedding_dim: int, num_embeddings: int, input_shape: Optional[Any] = None, num_target_parameters: int = 0):
        super().__init__(embedding_dim, num_embeddings, input_shape, num_target_parameters)
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)

    def forward(self, inp: Optional[Any] = None, *args, **kwargs):
        indices = torch.arange(self.num_embeddings).to(self.device)
        return self.embedding(indices), {}


### Weight Generator

In [4]:
class DeepTorchWeightGenerator(TorchWeightGenerator):
    def __init__(self, embedding_dim: int, num_embeddings: int, hidden_dim: int, input_shape: Optional[Any] = None, num_target_parameters: int = 0):
        super().__init__(embedding_dim, num_embeddings, hidden_dim, input_shape, num_target_parameters)
        self.linear1 = nn.Linear(embedding_dim, 32)
        self.linear2 = nn.Linear(32, hidden_dim)

    def forward(
        self, embedding: torch.Tensor, inp: Optional[Any] = None
    ) -> torch.Tensor:
        x = self.linear1(embedding)
        x = F.relu(x)
        return self.linear2(x).view(-1), {}



### Making a LunarLander HyperNetwork

#### Big Target Network

In [ ]:
target_network = nn.Sequential(
    nn.Linear(8, 256, bias=False),
    nn.Tanh(),
    nn.Linear(256,256, bias=False),
    nn.Tanh(),
    nn.Linear(256, 4, bias=False)
)
pytorch_total_params = sum(p.numel() for p in target_network.parameters() if p.requires_grad)
pytorch_total_params

In [ ]:
EMBEDDING_DIM = 32
NUM_EMBEDDINGS = 512

embedding_network = DefaultTorchEmbeddingModule.from_target(target_network, EMBEDDING_DIM, NUM_EMBEDDINGS)
weight_generator = DeepTorchWeightGenerator.from_target(target_network, EMBEDDING_DIM, NUM_EMBEDDINGS)

#### Much smaller Hypernetwork, with lots of parameter sharing

In [ ]:
hypernetwork = TorchHyperNetwork.from_target(
                            target_network,
                            embedding_module=embedding_network,
                            weight_generator=weight_generator,
                        )
pytorch_total_params = sum(p.numel() for p in hypernetwork.parameters() if p.requires_grad)
pytorch_total_params

In [ ]:
hypernetwork

#### Tensorboard logging

In [5]:
import os
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

def get_tensorboard_logger(
    experiment_name: str, base_log_path: str = "tensorboard_logs"
):
    log_path = "{}/{}_{}".format(base_log_path, experiment_name, datetime.now())
    train_writer = SummaryWriter(log_path, flush_secs=10)
    full_log_path = os.path.join(os.getcwd(), log_path)
    print(
        "Follow tensorboard logs with: python -m tensorboard.main --logdir '{}'".format(full_log_path)
    )
    return train_writer


In [6]:
def rollout(env, hypernetwork, render=False) -> float:
    with torch.no_grad():
        params, embedding, embedding_module_output, weight_generator_output = hypernetwork.generate_params()
        obs = env.reset()
        done = False
        observations, actions, rewards, rendereds = [], [], [], []
        while not done:
            rendered = None
            if render:
                rendered = env.render(mode="rgb_array")
                rendereds.append(rendered)

            action_logits = hypernetwork(inp=[torch.from_numpy(obs).unsqueeze(0).to(hypernetwork.device)], generated_params=params, has_aux=False)
            dist = Categorical(logits=action_logits)
            action = dist.sample().item()
            next_obs, r, done, _ = env.step(action)

            observations.append(obs)
            actions.append(action)
            rewards.append(r)

            obs = next_obs

    env.close()
    return observations, actions, rewards, rendereds

def discount_reward(rews, gamma: float = 0.99):
    n = len(rews)
    rtgs = np.zeros_like(rews)
    for i in reversed(range(n)):
        rtgs[i] = rews[i] + gamma*(rtgs[i + 1] if i + 1 < n else 0)
    return rtgs

def reinforce(
        num_epochs,
        env,
        hypernetwork,
        rollout_fn=rollout,
        lr: float = 0.0001,
        gamma: float = 0.99,
    ):

    writer = get_tensorboard_logger("HypernetworkTorchRL")
    optimizer = optim.Adam(hypernetwork.parameters(), lr=lr)

    bar = tqdm.tqdm(np.arange(num_epochs))
    for i in bar:
        observations, actions, rewards, _ = rollout_fn(env, hypernetwork)

        discounted_rewards = discount_reward(np.array(rewards), gamma)
        discounted_rewards = discounted_rewards - np.mean(discounted_rewards)
        discounted_rewards = discounted_rewards / (
            np.std(discounted_rewards) + 1e-10
        )

        observations = torch.from_numpy(np.array(observations)).float().to(hypernetwork.device)
        actions = torch.from_numpy(np.array(actions)).float().to(hypernetwork.device)
        discounted_rewards = torch.from_numpy(discounted_rewards).float().to(hypernetwork.device)

        logits, generated_params, embedding, embedding_module_output, weight_geneerator_module = hypernetwork(inp=[observations], has_aux=True)
        dist = Categorical(logits=logits)
        log_probs = dist.log_prob(actions)

        optimizer.zero_grad()
        loss = -1 * torch.sum(discounted_rewards * log_probs)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(hypernetwork.parameters(), 10.0)
        optimizer.step()

        grad_dict = {}
        for n, W in hypernetwork.named_parameters():
            if W.grad is not None:
                grad_dict["{}_grad".format(n)] = float(torch.sum(W.grad).item())

        metrics = {"loss":loss.item(), "rewards":np.sum(rewards), **grad_dict}

        for key in metrics:
            writer.add_scalar(key, metrics[key], i)

        bar.set_description('Loss: {}, Sum Reward: {}'.format(loss.item(), np.sum(rewards)))


In [ ]:
env = gym.make("LunarLander-v2")

In [ ]:
reinforce(100000, env, hypernetwork, lr=0.00002)

In [ ]:
env = gym.make("LunarLander-v2")
observations, actions, rewards, rendereds = rollout(env, hypernetwork, render=True)

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib ipympl

plt.style.use('ggplot')
# Imports specifically so we can render outputs in Jupyter.
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display
from celluloid import Camera
from IPython.display import HTML


def render_rollout(model):
    fig = plt.figure("Animation",figsize=(7,5))
    camera = Camera(fig)
    ax = fig.add_subplot(111)
    observations, actions, rewards, rendereds = rollout(gym.make("LunarLander-v2"), model, render=True)
    frames = []
    for r in rendereds:
        frame = ax.imshow(r)
        ax.axis('off')
        camera.snap()
        frames.append([frame])
    animation = camera.animate(blit=False, interval=50)
    # display(animations.to_html5_video())
    animation.save('LunarLander.mp4')
    return animation


In [ ]:
vid = render_rollout(hypernetwork)

In [ ]:
from IPython.display import Video

Video("LunarLander.mp4")

In [ ]:
param_vector, _, _ = hypernetwork.generate_params()

In [ ]:
params = []
start = 0
for p in hypernetwork._target.named_params:
    end = start + np.prod(p.size())
    params.append(param_vector[start:end].view(p.size()).detach().cpu().numpy())
    start = end

In [ ]:
[p.shape for p in params]

In [ ]:
f, (a0, a1) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [1, 1]})
a0.imshow(params[0])
a1.imshow(params[1])
f.tight_layout()


In [ ]:
import matplotlib.pyplot as plt

def plot_params(params):
    fig = plt.figure(figsize=(8, 8))
    num_params = len(params)
    for i in range(num_params):
        fig.add_subplot(1,num_params,i+1, )
        p = params[i]
        if len(p.shape) == 1:
            p = np.expand_dims(p,1)
        plt.imshow(p)
    fig.tight_layout()
    plt.show()

plot_params(params)


In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12, 12))
fig.add_subplot(1,2,1)
plt.imshow(params[0])
fig.add_subplot(1,2,2)
plt.imshow(params[1])
plt.subplots_adjust(wspace=0, hspace=0)
plt.tight_layout()
plt.show()



#### HyperHyperHyper

In [7]:
from typing import Optional, Any, Dict
NUM_LEVELS = 7

class DefaultTorchEmbeddingModule(TorchEmbeddingModule):
    def __init__(self, embedding_dim: int, num_embeddings: int, input_shape: Optional[Any] = None):
        super().__init__(embedding_dim, num_embeddings, input_shape)
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)

    def forward(self, inp: Optional[Any] = None, *args, **kwargs):
        indices = torch.arange(self.num_embeddings).to(self.device)
        return self.embedding(indices), {}

class DeepTorchWeightGenerator(TorchWeightGenerator):
    def __init__(self, embedding_dim: int, num_embeddings: int, hidden_dim: int, input_shape: Optional[Any] = None):
        super().__init__(embedding_dim, num_embeddings, hidden_dim, input_shape)
        self.linear1 = nn.Linear(embedding_dim, 32)
        self.linear2 = nn.Linear(32, hidden_dim, bias=False)

    def forward(
        self, embedding: torch.Tensor, inp: Optional[Any] = None
    ) -> torch.Tensor:
        x = self.linear1(embedding)
        x = torch.tanh(x)
        x = self.linear2(x).view(-1)
        x = x / x.max()
        # x = (x - x.mean()) / x.std()
        return x, {}



In [8]:
def hyper_hyper(num_levels: int, target_network, embedding_dim, num_embeddings):
    t = target_network
    for i in range(num_levels - 1):
        hypernetwork = TorchHyperNetwork.from_target(
                    t,
                    embedding_module=DefaultTorchEmbeddingModule,
                    weight_generator=DeepTorchWeightGenerator,
                    embedding_dim=embedding_dim,
                    num_embeddings=num_embeddings
        )
        t = hypernetwork
    hypernetwork = TorchHyperNetwork.from_target(
                t,
                embedding_module=DefaultTorchEmbeddingModule,
                weight_generator=DeepTorchWeightGenerator,
                embedding_dim=4,
                num_embeddings=num_embeddings
    )
    return hypernetwork

In [9]:
NUM_LEVELS = 7

In [10]:
target_network = nn.Sequential(
    nn.Linear(8, 256, bias=True),
    nn.Tanh(),
    nn.Linear(256,256, bias=True),
    nn.Tanh(),
    nn.Linear(256,256, bias=True),
    nn.Tanh(),
    nn.Linear(256,256, bias=True),
    nn.Tanh(),
    nn.Linear(256,256, bias=True),
    nn.Tanh(),
    nn.Linear(256, 4, bias=False)
)

recursive_hyper = hyper_hyper(NUM_LEVELS, target_network, 8, 128)

In [11]:
pytorch_total_params = sum(p.numel() for p in recursive_hyper.parameters() if p.requires_grad)
pytorch_total_params

1156

In [12]:
def rollout_recursive(env, hypernetwork, render=False) -> float:
    with torch.no_grad():
        # params, embedding, embedding_module_output, weight_generator_output = hypernetwork.generate_params()
        obs = env.reset()
        done = False
        observations, actions, rewards, rendereds = [], [], [], []
        while not done:
            rendered = None
            if render:
                rendered = env.render(mode="rgb_array")
                rendereds.append(rendered)

            inp = torch.from_numpy(obs).unsqueeze(0).to(hypernetwork.device)
            for i in range(NUM_LEVELS - 1):
                inp = [inp]

            action_logits = hypernetwork(inp=[inp], has_aux=False)
            dist = Categorical(logits=action_logits)
            action = dist.sample().item()
            next_obs, r, done, _ = env.step(action)

            observations.append(obs)
            actions.append(action)
            rewards.append(r)

            obs = next_obs

    env.close()
    return observations, actions, rewards, rendereds


In [13]:
def discount_reward(rews, gamma: float = 0.99):
    n = len(rews)
    rtgs = np.zeros_like(rews)
    for i in reversed(range(n)):
        rtgs[i] = rews[i] + gamma*(rtgs[i + 1] if i + 1 < n else 0)
    return rtgs

def reinforce_recursive(
        num_epochs,
        env,
        hypernetwork,
        rollout_fn=rollout_recursive,
        lr: float = 0.0001,
        gamma: float = 0.99,
    ):

    writer = get_tensorboard_logger("RecursiveHypernetworkTorchRL")
    optimizer = optim.Adam(hypernetwork.parameters(), lr=lr)

    bar = tqdm.tqdm(np.arange(num_epochs))
    for i in bar:
        observations, actions, rewards, _ = rollout_fn(env, hypernetwork)

        discounted_rewards = discount_reward(np.array(rewards), gamma)
        discounted_rewards = discounted_rewards - np.mean(discounted_rewards)
        discounted_rewards = discounted_rewards / (
            np.std(discounted_rewards) + 1e-10
        )

        observations = torch.from_numpy(np.array(observations)).float().to(hypernetwork.device)
        actions = torch.from_numpy(np.array(actions)).float().to(hypernetwork.device)
        discounted_rewards = torch.from_numpy(discounted_rewards).float().to(hypernetwork.device)

        inp = observations
        for _ in range(NUM_LEVELS - 1):
            inp = [inp]

        logits = hypernetwork(inp=[inp], has_aux=False)
        dist = Categorical(logits=logits)
        log_probs = dist.log_prob(actions)

        optimizer.zero_grad()
        loss = -1 * torch.sum(discounted_rewards * log_probs)

        loss.backward()
        for n, W in hypernetwork.named_parameters():
            if W.grad is not None:
                W.grad = W.grad / W.grad.norm()
        # torch.nn.utils.clip_grad_norm_(hypernetwork.parameters(), 1.0)
        optimizer.step()

        grad_dict = {}
        for n, W in hypernetwork.named_parameters():
            if W.grad is not None:
                grad_dict["{}_grad".format(n)] = float(torch.sum(W.grad).item())

        metrics = {"loss":loss.item(), "rewards":np.sum(rewards), **grad_dict}

        for key in metrics:
            writer.add_scalar(key, metrics[key], i)

        bar.set_description('Loss: {}, Sum Reward: {}'.format(loss.item(), np.sum(rewards)))


In [17]:
NUM_LEVELS = 5

target_network = nn.Sequential(
    nn.Linear(8, 256, bias=True),
    nn.Tanh(),
    nn.Linear(256,256, bias=True),
    nn.Tanh(),
    nn.Linear(256,256, bias=True),
    nn.Tanh(),
    nn.Linear(256,256, bias=True),
    nn.Tanh(),
    nn.Linear(256,256, bias=True),
    nn.Tanh(),
    nn.Linear(256, 4, bias=False)
)

recursive_hyper = hyper_hyper(NUM_LEVELS, target_network, 8, 128)

In [18]:
pytorch_total_params = sum(p.numel() for p in recursive_hyper.parameters() if p.requires_grad)
pytorch_total_params

1380

In [19]:
env = gym.make("LunarLander-v2")

reinforce_recursive(100000, env, recursive_hyper, rollout_fn=rollout_recursive, lr=0.00001)

Follow tensorboard logs with: python -m tensorboard.main --logdir '/home/shyam/Code/hyper-nn/notebooks/torch/tensorboard_logs/RecursiveHypernetworkTorchRL_2022-04-24 20:55:32.397401'


Loss: -4.800282001495361, Sum Reward: -150.61092547317915:  20%|█▉        | 19964/100000 [1:28:43<5:55:42,  3.75it/s]   


KeyboardInterrupt: 